# CSE258 Assignment 1

### nHelpful

In [3]:
import gzip
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt
from collections import defaultdict

In [4]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def readJson(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return df

df = readJson('assignment1/train.json.gz')

In [47]:
Data_total = [df[x] for x in range(0,len(df))]
Data_train = [df[x] for x in range(0,len(df)/2)]
Data_valid = [df[x] for x in range(len(df)/2,len(df))]

In [48]:
df2 = readJson('assignment1/test_Helpful.json.gz')
Data_test = [df2[x] for x in range(0,len(df2))]

In [234]:
# item average rating
rateNum = {}
rateAvg = {}
globalAvg = sum([x['rating'] for x in Data_total]) * 1.0 / len([x['rating'] for x in Data_total])
for x in Data_total:
    if x['itemID'] in rateAvg:
        rateAvg[x['itemID']] = (rateAvg[x['itemID']] * rateNum[x['itemID']] + x['rating']) / (rateNum[x['itemID']] + 1)
        rateNum[x['itemID']] += 1
    else:
        rateNum[x['itemID']] = 1
        rateAvg[x['itemID']] = x['rating']

In [235]:
# number of items user reviewed
reviewNum = defaultdict(int)
for x in Data_total:
    reviewNum[x['reviewerID']] += 1

In [236]:
# variance of item rating
itemRateList = defaultdict(list)
itemRateVar = defaultdict(float)
for x in Data_total:
    itemRateList[x['reviewerID']].append(x['rating'])
for x in itemRateList:
    itemRateVar[x] = np.std(itemRateList[x])

In [252]:
# item total outOf
itemOutOf = defaultdict(int)
for x in Data_total:
    itemOutOf[x['itemID']] += x['helpful']['outOf']

In [237]:
categoryIndex = ['Men', 'Women', 'Boys', 'Girls', 'Baby']

def encodeCategory(data, cat):
    for category in data['categories']:
        for subcategory in category:
            if subcategory == categoryIndex[cat]:
                return 1
    return 0

In [256]:
# nHelpful feature extraction
def featHelpful(data):
    votes = data['helpful']['outOf']
    
    feat = [1]
#     feat.append(data['rating'])
    feat.append(data['rating'] - globalAvg)
    feat.append(len(data['reviewText'].split( )) / data['rating'])
    
    feat.append(reviewNum[data['reviewerID']])
    feat.append(itemRateVar[data['itemID']])
    
    if data['itemID'] in rateAvg:
        feat.append(abs(data['rating'] - rateAvg[data['itemID']]))
    else:
        feat.append(0)
        
    feat.append('but' in data['reviewText'] or 'however' in data['reviewText'])
#     feat.append('!' in data['reviewText'])
#     feat.append('?' in data['reviewText'])
    feat.append(data['reviewText'].count('?') * 1.0 / len(data['reviewText'].split('.')))
    feat.append(data['reviewText'].count('!') * 1.0 / len(data['reviewText'].split('.')))
    
    feat.append(encodeCategory(data, 0))
    feat.append(encodeCategory(data, 1))
    feat.append(encodeCategory(data, 2))
    feat.append(encodeCategory(data, 3))
    feat.append(encodeCategory(data, 4))

    return feat

In [257]:
x1_train = [featHelpful(x) for x in Data_train if x['helpful']['outOf'] > 39]
y1 = [x['helpful']['nHelpful'] * 1.0 / x['helpful']['outOf'] for x in Data_train if x['helpful']['outOf'] > 39]
result1 = np.linalg.lstsq(x1_train, y1)
theta1 = result1[0]

y_valid = [x['helpful']['nHelpful'] for x in Data_valid]
pred_valid = []
for x in Data_valid:
    x_valid = featHelpful(x)
    pred_valid.append(np.round(min(np.dot(np.array(x_valid), theta1),1.0) * x['helpful']['outOf']))
print mean_absolute_error(pred_valid, y_valid), theta1

0.17435 [  7.48707188e-01   2.08980693e-02   3.32208805e-04  -1.97771402e-04
  -2.22044605e-16  -3.74975844e-02   9.24935246e-03  -2.79164671e-01
  -4.89857964e-03   1.52225458e-01   2.01505077e-01   2.02914875e-01
   1.88398511e-01   2.41325586e-01]


In [255]:
x1_test = [featHelpful(x) for x in Data_test]
pred_rate1 = np.dot(np.array(x1_test), theta1)

index = 0
predictions = open("assignment1/predictions_Helpful.txt", 'w')
for l in open("assignment1/pairs_Helpful.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,i,outOf = l.strip().split('-')
    outOf = int(outOf)
    predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(np.round(outOf * min(pred_rate1[index],1.0))) + '\n')
    index += 1

predictions.close()